In [ ]:
from CoRe_Dataloader_ECSG import get_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
trainds = get_dataset(rsamp = .8)
testds = get_dataset(rsamp = .2)
train_dl = DataLoader(trainds,batch_size=8,shuffle = True,)
test_ds = DataLoader(testds,batch_size=8,shuffle = True,)

In [ ]:
class LDB(nn.Module):
    def __init__(self,inD,outD,dropout) -> None:
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(inD,outD),
            nn.Dropout1d(p = dropout),
            nn.BatchNorm1d(outD)
        )
    def forward(self,x):
        return self.seq(x)
l = LDB(10,10,.5)
# l(torch.randn(16,10))

In [ ]:
class cog(nn.Module):
    def __init__(self,x,y,out_classes,device = "cpu") -> None:
        super().__init__()
        self.norm1 = nn.BatchNorm2d(1).to(device)
        self.flatten = nn.Flatten().to(device)
        self.l1 = LDB(x*y,4096,dropout = .2).to(device)
        self.l2 = LDB(4096,4096/2,.2).to(device)
        self.l3 = LDB(4096/2,out_classes,.2).to(device)
        self.seq = nn.Sequential(self.l1,self.l2,self.l3)
        self.softmax = nn.Softmax(-1).to(device)
    def forward(self,x: torch.Tensor):
        xs = x.shape
        ix = x.view((xs[0],1,xs[1],xs[2]))
        ix = self.norm1(ix)
        ix = self.flatten(ix)
        ix = self.seq(ix)
        return self.softmax(ix)

In [ ]:
model = cog(400,400,19,"cuda:0")
ti = torch.randn(16,400,400).to("cuda:0")
# model(ti)

In [ ]:
import torchinfo
torchinfo.summary(model,(1,400,400))

In [ ]:
import wandb
wandb.init("sdisc-2d")
config = wandb.config
config.epochs = 100
config.inx = 400
config.iny = 400
optimizer = optim.Adam(params = model.parameters(),lr = 1e-5)
lossfn = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1,config.epochs+2):
    ldl = len(train_dl)
    for batch,(sg,params) in enumerate(train_dl):
        sg = sg.to("cuda:0").to(torch.float)
        params = params[:,0].to("cuda:0").to(torch.long)
        optimizer.zero_grad()
        outputs = model(sg)
        loss = lossfn(outputs,params)
        loss.backward()
        optimizer.step()
        print(epoch,batch,loss.item())
